本NoteBook介绍了BERT相关模型和知识蒸馏过程

数据集：CoLL2003,分类数据集

先查看不同参数量的模型bert-base-uncased、DistilBERT、TinyBERT模型在该数据集上的表现情况；

此外，分析了两种学生模型的情况：
- 预训练模型TinyBERT作为学生模型时，模型的表现情况
- 自定义BERT模型结构作为学生模型，选择DistilBERT模型的部分层参数，然后观察表现情况

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
from transformers import BertTokenizer,BertModel

In [ ]:
current_tag2id = {
    'O': 0,
    'B-PER': 1,
    'I-PER': 2,
    'B-ORG': 3,
    'I-ORG': 4,
    'B-LOC': 5,
    'I-LOC': 6,
    'B-MISC': 7,
    'I-MISC': 8
}

In [ ]:
def parse_conll2003(file_path):
    sentences, labels = [], []
    sentence, label = [], []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip() == '':
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                    sentence, label = [], []
            else:
                token, _, _, tag = line.strip().split()
                sentence.append(token)
                label.append(tag)
    return sentences, labels

In [ ]:
file_path = "../content/test.txt"
sentences,labels = parse_conll2003(file_path)
sentences_test = sentences[:1500]
labels_test = labels[:1500]

In [ ]:
file_path = "../content/train.txt"
sentences,labels = parse_conll2003(file_path)#len(sentences) = 14987
sentences[1],labels[1]

(['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
 ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'])

In [ ]:
class NERDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len=128, current_tag2id=current_tag2id):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.current_tag2id = current_tag2id or {'O': 0}

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]
        tokens = []
        label_ids = []
        for word, tag in zip(sentence, label):
            word_tokens = self.tokenizer.tokenize(word)
            if word_tokens:
                tokens.extend(word_tokens)
                label_ids.extend([self.current_tag2id.get(tag, 0)] + [-100] * (len(word_tokens) - 1))
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        label_ids = [-100] + label_ids + [-100]
        if len(tokens) > self.max_len:
            tokens = tokens[:self.max_len]
            label_ids = label_ids[:self.max_len]
        else:
            padding_len = self.max_len - len(tokens)
            tokens += ['[PAD]'] * padding_len
            label_ids += [-100] * padding_len
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1 if token != '[PAD]' else 0 for token in tokens]
        return {
            'input_ids': torch.tensor(input_ids),
            'attention_mask': torch.tensor(attention_mask),
            'labels': torch.tensor(label_ids)
        }

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
Dataset = NERDataset(sentences,labels,tokenizer)

In [ ]:
from torch.utils.data import DataLoader

data_loader = DataLoader(Dataset,batch_size=32,shuffle=True)
for loader in data_loader:
    a = loader["input_ids"]
    print(len(a))
    break
len(data_loader)

32


469

In [ ]:
Dataset_test = NERDataset(sentences_test,labels_test,tokenizer)
data_loader_test = DataLoader(Dataset_test,batch_size=32,shuffle=True)

In [ ]:
class BertNER(nn.Module):
    def __init__(self, model,num_labels):
        super(BertNER, self).__init__()
        self.bert = model
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.num_labels = num_labels

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)#如果是10类，这个logits是向量？logits是未归一化的原始值，值可正可负
        return logits

    def extend_output_layer(self, new_labels):
        # 动态扩展输出层以适应新实体类型
        new_num_labels = self.num_labels + len(new_labels)
        old_classifier = self.classifier
        self.classifier = nn.Linear(self.bert.config.hidden_size, new_num_labels)
        with torch.no_grad():
            self.classifier.weight[:old_classifier.weight.size(0)] = old_classifier.weight
            self.classifier.bias[:old_classifier.bias.size(0)] = old_classifier.bias
        self.num_labels = new_num_labels

In [ ]:
from torch.nn.functional import kl_div, log_softmax, softmax

def train_with_distillation(model, teacher_model, dataloader, optimizer, device, alpha=0.3, T_m=2.0):
    model.train()
    if teacher_model:
        teacher_model.eval()
    total_loss = 0
    # student_loss = []
    # teacher_loss = []
    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    for step, batch in enumerate(dataloader, 1):#len(dataloader) = 469
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        logits = model(input_ids, attention_mask)
        ce_loss = criterion(logits.view(-1, model.num_labels), labels.view(-1))
        if teacher_model:
            with torch.no_grad():
                teacher_logits = teacher_model(input_ids, attention_mask)
            kl_loss = kl_div(log_softmax(logits / T_m, dim=-1),
                            softmax(teacher_logits / T_m, dim=-1),
                            reduction='batchmean') * (T_m ** 2)
            loss = alpha * kl_loss + (1-alpha) * ce_loss
            # student_loss.append(loss.item())
        else:
            loss = ce_loss
            # teacher_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()#loss是张量，张量转标量，然后相加
        if step == 1 or step % 50 == 0 or step == len(dataloader):
          print(f"[{step}/{len(dataloader)}] Loss: {loss.item():.5f}")
    return total_loss / len(dataloader)

In [ ]:
torch.cuda.is_available()

True

📌训练一个大模型（bert-base-uncased模型）作为教师模型，效果是怎么样的呢？

In [ ]:
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model = BertModel.from_pretrained('bert-base-uncased')
teacher = BertNER(bert_model,num_labels=9).to(device)#加载预训练模型
optimizer = optim.Adam(teacher.parameters(), lr=2e-5)
print(next(teacher.parameters()).device)

cuda:0


In [ ]:
avg_loss_teacher = train_with_distillation(
    model=teacher,
    teacher_model=None,        # 传 None
    dataloader=data_loader,
    optimizer=optimizer,
    device=device,
    alpha=0.3,
    T_m=2.0
)
avg_loss_teacher

[1/469] Loss: 2.32392
[50/469] Loss: 0.37947
[100/469] Loss: 0.18251
[150/469] Loss: 0.13941
[200/469] Loss: 0.08540
[250/469] Loss: 0.05417
[300/469] Loss: 0.07554
[350/469] Loss: 0.12813
[400/469] Loss: 0.03128
[450/469] Loss: 0.06368
[469/469] Loss: 0.10199


0.18652058681890146

In [ ]:
model_size = sum(param.numel() * param.element_size() for param in teacher.parameters())
print(f"Teacher Model【bert-base-uncased】 size: {model_size/(1024**2):.2f}MB")

Teacher Model【bert-base-uncased】 size: 417.67MB


In [ ]:
total_loss_teacher = 0
criterion = nn.CrossEntropyLoss(ignore_index=-100)
for step, batch in enumerate(data_loader_test, 1):#len(dataloader) = 469
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  labels = batch['labels'].to(device)
  logits = teacher(input_ids, attention_mask)
  ce_loss = criterion(logits.view(-1, teacher.num_labels), labels.view(-1))
  total_loss_teacher += ce_loss.item()
total_loss_teacher

4.037542760372162

可以看到【bert-base-uncased】：
- 训练时长总共花了5分钟，
- 教师模型的参数量大小为417.67MB，
- 最终的`avg_loss_teacher`为0.186左右
- 测试集损失在4.037左右

📌如果我直接训练一个小模型（distilbert模型），效果是怎么样的呢？

In [ ]:
import torch.optim as optim
from transformers import DistilBertModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DistilBERT_Student_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
distilbert = BertNER(DistilBERT_Student_model,num_labels=9).to(device)#加载预训练模型
optimizer = optim.Adam(distilbert.parameters(), lr=2e-5)
avg_loss_distilbert = train_with_distillation(
    model=distilbert,
    teacher_model=None,        # 传 None
    dataloader=data_loader,
    optimizer=optimizer,
    device=device,
    alpha=0.3,
    T_m=2.0
)
avg_loss_distilbert

[1/469] Loss: 2.34545
[50/469] Loss: 0.45150
[100/469] Loss: 0.27685
[150/469] Loss: 0.09157
[200/469] Loss: 0.05214
[250/469] Loss: 0.05519
[300/469] Loss: 0.12074
[350/469] Loss: 0.11109
[400/469] Loss: 0.06805
[450/469] Loss: 0.03243
[469/469] Loss: 0.04553


0.2081779987334824

In [ ]:
# model_size = sum(param.numel() * param.element_size() for param in distilbert.parameters())
# print(f"Distilbert Model size: {model_size/(1024**2):.2f}MB")

In [ ]:
#查看一下模型层数，因为待会儿（最后部分）我们要提取第1层，第3层，第6层,即[0,2,5]
print(f"Encoder 层数（from config）: {DistilBERT_Student_model.config.num_hidden_layers}")

Encoder 层数（from config）: 6


In [ ]:
total_loss_teacher = 0
criterion = nn.CrossEntropyLoss(ignore_index=-100)
for step, batch in enumerate(data_loader_test, 1):#len(dataloader) = 469
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  labels = batch['labels'].to(device)
  logits = distilbert(input_ids, attention_mask)
  ce_loss = criterion(logits.view(-1, distilbert.num_labels), labels.view(-1))
  total_loss_teacher += ce_loss.item()
total_loss_teacher

4.370914436876774

可以看到【Distilbert】：
- 训练时长总共花了2分钟，
- 学生模型的参数量大小为253.18MB，
- 最终的`avg_loss_distilbert`为0.21左右
- 测试损失在4.371左右

📌如果我直接训练一个小模型（TinyBert模型），效果是怎么样的呢？

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
Tinybert = BertNER(model,num_labels=9).to(device)#加载预训练模型

optimizer = optim.Adam(Tinybert.parameters(), lr=2e-5)
avg_loss_Tinybert = train_with_distillation(
    model=Tinybert,
    teacher_model=None,        # 传入 None
    dataloader=data_loader,
    optimizer=optimizer,
    device=device,
    alpha=0.3,
    T_m=2.0
)
avg_loss_Tinybert

config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

[1/469] Loss: 2.36529


model.safetensors:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

[50/469] Loss: 0.81826
[100/469] Loss: 0.47840
[150/469] Loss: 0.27166
[200/469] Loss: 0.32811
[250/469] Loss: 0.27256
[300/469] Loss: 0.24039
[350/469] Loss: 0.33048
[400/469] Loss: 0.16925
[450/469] Loss: 0.17254
[469/469] Loss: 0.19439


0.35843750121179163

In [ ]:
model_size = sum(param.numel() * param.element_size() for param in Tinybert.parameters())
print(f"Tinylbert Model size: {model_size/(1024**2):.2f}MB")

Tinylbert Model size: 54.75MB


In [ ]:
#模型在测试集上的效果
total_loss = 0
criterion = nn.CrossEntropyLoss(ignore_index=-100)
for step, batch in enumerate(data_loader_test, 1):#len(dataloader) = 469
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  labels = batch['labels'].to(device)
  logits = Tinybert(input_ids, attention_mask)
  ce_loss = criterion(logits.view(-1, Tinybert.num_labels), labels.view(-1))
  total_loss += ce_loss.item()
total_loss

9.416891254484653

可以看到【TinyBert】：
- 训练时长总共花了32秒，
- 学生模型的参数量大小为54.75MB，
- 最终的`avg_loss_TinyBert`为0.358左右
- 最终测试集损失在9.417

📌好，接下来咱们开始蒸馏，来查看一下使用`TinyBert`作为BaseLine得到的学生模型是不是要比直接训练`TinyBert`的效果要好

In [ ]:
model = AutoModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
Tinybert_Student = BertNER(model,num_labels=9).to(device)#加载预训练模型

In [ ]:
optimizer = optim.Adam(Tinybert_Student.parameters(), lr=2e-5)

avg_loss_student = train_with_distillation(
    model=Tinybert_Student,
    teacher_model=teacher,
    dataloader=data_loader,
    optimizer=optimizer,
    device=device,
    alpha=0.3,
    T_m=2.0
)


[1/469] Loss: 10.54618
[50/469] Loss: 12.98627
[100/469] Loss: 13.38211
[150/469] Loss: 15.60458
[200/469] Loss: 13.52334
[250/469] Loss: 16.46437
[300/469] Loss: 12.29873
[350/469] Loss: 10.20807
[400/469] Loss: 11.75377
[450/469] Loss: 14.29638
[469/469] Loss: 8.14688


In [ ]:
#模型在测试集上的效果
total_loss = 0
criterion = nn.CrossEntropyLoss(ignore_index=-100)
for step, batch in enumerate(data_loader_test, 1):#len(dataloader) = 469
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  labels = batch['labels'].to(device)
  logits = Tinybert_Student(input_ids, attention_mask)
  ce_loss = criterion(logits.view(-1, Tinybert_Student.num_labels), labels.view(-1))
  total_loss += ce_loss.item()
total_loss

9.209900744259357

可以看到【Tinybert_Student】：
- 蒸馏时长花了2分钟，
- 学生模型的参数量大小依然为54.75MB（和TinyBert一样）
- 最终测试集损失在9.2

📌好，接下来咱们换一个学生模型，将Teacher模型的部分层【也就是0、2、5层】的权重通过深拷贝，赋值给自定义的学生模型

In [ ]:
import copy
from transformers import DistilBertModel, DistilBertConfig

# === 2. 创建学生模型配置（6层） ===
student_config = copy.deepcopy(DistilBERT_Student_model.config)
student_config.num_hidden_layers = 3
student_model = DistilBertModel(student_config)

# === 4. 选取教师模型中的层（每隔一层）拷贝到学生模型 ===
selected_teacher_layers = [0, 2, 5]

for student_layer_idx, teacher_layer_idx in enumerate(selected_teacher_layers):
    student_model.transformer.layer[student_layer_idx].load_state_dict(
        DistilBERT_Student_model.transformer.layer[teacher_layer_idx].state_dict()
    )



In [ ]:
distilbert_student = BertNER(student_model,num_labels=9).to(device)#加载预训练模型
model_size = sum(param.numel() * param.element_size() for param in distilbert_student.parameters())
print(f"Distilbert Model size: {model_size/(1024**2):.2f}MB")


Distilbert Model size: 172.07MB


In [ ]:
optimizer = optim.Adam(distilbert_student.parameters(), lr=2e-4)
avg_loss_distilbert = train_with_distillation(
    model=distilbert_student,
    teacher_model=distilbert,        # 传 None
    dataloader=data_loader,
    optimizer=optimizer,
    device=device,
    alpha=0.3,
    T_m=2.0
)

[1/469] Loss: 34.72672
[50/469] Loss: 37.75502
[100/469] Loss: 34.44826
[150/469] Loss: 32.47401
[200/469] Loss: 30.43495
[250/469] Loss: 24.58235
[300/469] Loss: 17.48047
[350/469] Loss: 20.76854
[400/469] Loss: 19.20379
[450/469] Loss: 14.74362
[469/469] Loss: 18.37824


In [ ]:
#模型在测试集上的效果
total_loss = 0
criterion = nn.CrossEntropyLoss(ignore_index=-100)
for step, batch in enumerate(data_loader_test, 1):#len(dataloader) = 469
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  labels = batch['labels'].to(device)
  logits = distilbert_student(input_ids, attention_mask)
  ce_loss = criterion(logits.view(-1, distilbert_student.num_labels), labels.view(-1))
  total_loss += ce_loss.item()
total_loss

21.31545290350914